In [1]:
import calib
from calib import board_points
import glob, os
import cv2
from i308_utils import imshow, show_images
import pickle
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d

# Calibración

In [2]:
checkerboard = (10, 7)
square_size_mm = 24.2

# creamos los puntos del mundo del objeto checkerboard
checkerboard_world_points_mm = board_points(checkerboard) * square_size_mm

directory = os.path.join("datasets", "buddha_board", "calib")
left_files_pattern = "*left*.jpg"
right_files_pattern = "*right*.jpg"

def numeric_sort(file_name):
    return int(file_name.split("_")[-1].split(".")[0])

left_file_names = sorted(
    glob.glob(
        os.path.join(directory, left_files_pattern)
    ),
    key=numeric_sort
)

right_file_names = sorted(
    glob.glob(
        os.path.join(directory, right_files_pattern)
    ),
    key=numeric_sort
)

num_left = len(left_file_names)
num_right = len(right_file_names)

if  num_left != num_right:
    raise Exception(f"the number of files (left {num_left} / right{num_right}) doesn't match")

In [3]:
image_size = None
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-3)

world_points = []
left_images = []
right_images = []
left_images_points = []
right_images_points = []

for left_file_name, right_file_name in zip(
    left_file_names, right_file_names
):
    # read left and right images
    left_image = cv2.imread(left_file_name, cv2.IMREAD_GRAYSCALE)
    right_image = cv2.imread(right_file_name, cv2.IMREAD_GRAYSCALE)

    # get the images sizes
    left_size = (left_image.shape[1], left_image.shape[0])
    right_size = (right_image.shape[1], right_image.shape[0])

    # checks that images sizes match
    if left_size != right_size:
        raise Exception(f"left and right images sizes differ: left {left_size} / right {right_size}")
        
    if image_size is None:
        # remembers the images size
        image_size = left_size
    else:
        if image_size != left_size:
            raise Exception(f"there are images with different sizes: {image_size} vs {left_size}")

    # finds the checkerboard in each image
    left_found, left_corners = cv2.findChessboardCorners(left_image, checkerboard)
    right_found, right_corners = cv2.findChessboardCorners(right_image, checkerboard)

    if not left_found or not right_found:
        print("warning, checkerboard was not found")
        continue

    # checkerboard was found in both images.

    # let's improve the found corners
    corners_left = cv2.cornerSubPix(left_image, left_corners, (7, 7), (-1,-1), criteria)
    corners_right = cv2.cornerSubPix(right_image, right_corners, (7, 7), (-1,-1), criteria)

    # acumulo las imagenes
    left_images.append(left_image)
    right_images.append(right_image)

    # acumulo los corners detectados
    left_images_points.append(left_corners)
    right_images_points.append(right_corners)

    # acumulo los puntos del mundo
    world_points.append(checkerboard_world_points_mm)

In [4]:
err, left_K, left_dist, right_K, right_dist, R, T, E, F = cv2.stereoCalibrate(
    world_points, 
    left_images_points, 
    right_images_points, 
    None, 
    None, 
    None, 
    None, 
    image_size, 
    flags=0
)

# serializamos los resultados en un pickle

calibration_results = {
    'left_K': left_K,
    'left_dist': left_dist,
    'right_K': right_K,
    'right_dist': right_dist,
    'R': R,
    'T': T,
    'E': E,
    'F': F,
    'image_size': image_size,
}

calibration_file = os.path.join("datasets", "buddha_board", "stereo_calibration.pkl")
with open(calibration_file, "wb") as f:
    f.write(pickle.dumps(calibration_results))

In [5]:
# para recuperar los resultados podemos cargarlos usando pickle:

with open(calibration_file, "rb") as f:
    results2 = pickle.loads(f.read())

# Rectificación Estéreo

In [6]:
# realicemos la rectificación estéreo:
R1, R2, P1, P2, Q, validRoi1, validRoi2 = cv2.stereoRectify(
    left_K, left_dist, right_K, right_dist, image_size, R, T, alpha=0
)

# calculemos los mapas de des-distorsión-rectificación:
left_map_x, left_map_y = cv2.initUndistortRectifyMap(left_K, left_dist, R1, P1, image_size, cv2.CV_32FC1)
right_map_x, right_map_y = cv2.initUndistortRectifyMap(right_K, right_dist, R2, P2, image_size, cv2.CV_32FC1)

# Guardemos los resultados de rectificación para usar posteriormente:
stereo_maps = {

    # undistorting maps
    "left_map_x": left_map_x,
    "left_map_y": left_map_y,
    "right_map_x": right_map_x,
    "right_map_y": right_map_y,

    # add also rectifying info:
    "R1": R1,
    "R2": R2,
    "P1": P1,
    "P2": P2,
    "Q": Q,
    "validRoi1": validRoi1,
    "validRoi2": validRoi2,
}

stereo_maps_file = os.path.join("datasets", "buddha_board", "stereo_maps.pkl")
with open(stereo_maps_file, "wb") as f:
    f.write(pickle.dumps(stereo_maps))

In [7]:
directory = os.path.join("datasets", "buddha_board", "captures")
left_files_pattern = "left*.jpg"
right_files_pattern = "right*.jpg"

left_images= []
right_images= []

def numeric_sort(file_name):
    return int(file_name.split("_")[-1].split(".")[0])

left_file_names = sorted(
    glob.glob(
        os.path.join(directory, left_files_pattern)
    ),
    key=numeric_sort
)

right_file_names = sorted(
    glob.glob(
        os.path.join(directory, right_files_pattern)
    ),
    key=numeric_sort
)

for left_file_name, right_file_name in zip(
    left_file_names, right_file_names
):
    # read left and right images
    left_image = cv2.imread(left_file_name, cv2.IMREAD_GRAYSCALE)
    right_image = cv2.imread(right_file_name, cv2.IMREAD_GRAYSCALE)

    # acumulo las imagenes
    left_images.append(left_image)
    right_images.append(right_image)

In [8]:
with open("datasets/buddha_board/stereo_calibration.pkl", "rb") as f:
    calib = pickle.load(f)

with open("datasets/buddha_board/stereo_maps.pkl", "rb") as f:
    maps = pickle.load(f)

K1 = calib["left_K"]
D1 = calib["left_dist"]
Q = maps["Q"]
left_map_x = maps["left_map_x"]
left_map_y = maps["left_map_y"]
right_map_x = maps["right_map_x"]
right_map_y = maps["right_map_y"]

In [9]:
len(left_images)

6

In [11]:
left_rectified_imgs = []
right_rectified_imgs = []
disparity_maps = []

stereo = cv2.StereoSGBM_create(
    numDisparities=128,
    blockSize=7,
    P1=16,
    P2=128,
    uniquenessRatio=7,
    speckleWindowSize=512,
    speckleRange=64,
    disp12MaxDiff=1,
    preFilterCap=63
)

for i in range(len(left_images)):
    left_img  = left_images[i]
    right_img = right_images[i]

    # 1) Undistort & rectify
    left_rectified  = cv2.remap(left_img,  left_map_x,  left_map_y,  cv2.INTER_LINEAR)
    right_rectified = cv2.remap(right_img, right_map_x, right_map_y, cv2.INTER_LINEAR)
    left_rectified_imgs.append(left_rectified)
    right_rectified_imgs.append(right_rectified)

    # 2) Disparity
    disp = stereo.compute(left_rectified, right_rectified).astype(np.float32) / 16.0
    disparity_maps.append(disp)

    # 3) Reproyección 3D + filtrado de puntos inválidos
    pts_3d = cv2.reprojectImageTo3D(disp, Q)            # shape (H, W, 3)
    mask3d = (disp > 0) & np.isfinite(pts_3d).all(axis=2)
    output_points = pts_3d[mask3d]                      # (N_valid, 3)
    output_colors = cv2.cvtColor(left_rectified,
                                 cv2.COLOR_BGR2RGB)[mask3d]  # (N_valid, 3)

    # 4) Detección de checkerboard en la cara izquierda rectificada
    found, corners = cv2.findChessboardCorners(left_rectified, (10, 7))
    if not found:
        print(f"Checkerboard no detectado en imagen {i}")
        continue

    # justo antes de cornerSubPix:
    if left_rectified.ndim == 3:  
        # imagen color → convierto a gris
        gray = cv2.cvtColor(left_rectified, cv2.COLOR_BGR2GRAY)
    else:
        # ya es single–channel
        gray = left_rectified
        
    corners_subpix = cv2.cornerSubPix(
        gray, corners,
        winSize=(11,11), zeroZone=(-1,-1),
        criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.01)
    )

    # 5) SolvePnP → pose_matrix
    retval, rvec, tvec = cv2.solvePnP(
        checkerboard_world_points_mm, corners_subpix, K1, D1
    )
    R_w, _ = cv2.Rodrigues(rvec)
    T_w = tvec.flatten()
    pose_matrix = np.eye(4)
    pose_matrix[:3, :3] = R_w
    pose_matrix[:3,  3] = T_w

    # 6) Transformación homogénea de los puntos válidos
    ones = np.ones((output_points.shape[0], 1))
    homog = np.hstack([output_points, ones])           # (N_valid, 4)
    pts_world = (pose_matrix @ homog.T).T[:, :3]       # (N_valid, 3)

    # 7) Construcción del PointCloud en Open3D
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pts_world)
    pcd.colors = o3d.utility.Vector3dVector(output_colors / 255.0)

    # 8) Marcos de referencia
    world_frame  = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1)
    camera_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1)
    camera_frame.transform(pose_matrix)


    # 10) Visualizar
    # o3d.visualization.draw_geometries(
    #     [pcd_cropped, world_frame, camera_frame])

# Calavera (seguir después)

In [ ]:
directory = os.path.join("datasets", "skull_charuco", "calib")
left_files_pattern = "*left*.jpg"
right_files_pattern = "*right*.jpg"
left_images = []
right_images = []

def numeric_sort(file_name):
    return int(file_name.split("_")[-1].split(".")[0])

left_file_names = sorted(
    glob.glob(
        os.path.join(directory, left_files_pattern)
    ),
    key=numeric_sort
)

right_file_names = sorted(
    glob.glob(
        os.path.join(directory, right_files_pattern)
    ),
    key=numeric_sort
)   

for left_file_name, right_file_name in zip(
    left_file_names, right_file_names
):
    left_image = cv2.imread(left_file_name, cv2.IMREAD_GRAYSCALE)
    right_image = cv2.imread(right_file_name, cv2.IMREAD_GRAYSCALE)
    left_images.append(left_image)
    right_images.append(right_image)

: 

In [ ]:
img_num = 27

left_image_file = os.path.join("datasets", "skull_charuco", "captures", f"left_{img_num}.jpg")
right_image_file = os.path.join("datasets", "skull_charuco", "captures", f"right_{img_num}.jpg")

left_image = cv2.imread(left_image_file, 0)
right_image = cv2.imread(right_image_file, 0)

: 

In [ ]:
stereo_calib_file = os.path.join("datasets", "skull_charuco", "stereo_calibration.pkl")
stereo_maps_file = os.path.join("datasets", "skull_charuco", "stereo_maps.pkl")

print("reading stereo calibration results...")
with open(stereo_calib_file, "rb") as f:
    calibration = pickle.loads(f.read())
    
print("reading stereo rectification maps...")
with open(stereo_maps_file, "rb") as f:
    maps = pickle.loads(f.read())

: 

In [ ]:
left_map_x, left_map_y = maps['left_map_x'], maps['left_map_y']
right_map_x, right_map_y = maps['right_map_x'], maps['right_map_y']



left_image_rectified = cv2.remap(left_image, left_map_x, left_map_y, cv2.INTER_LINEAR)
right_image_rectified = cv2.remap(right_image, right_map_x, right_map_y, cv2.INTER_LINEAR)

fig, axes = show_images([
    left_image_rectified, right_image_rectified
], [
    "left rectified", "right rectified"
], show=False)

for y, c in zip([255, 470, 830], ['r', 'b', 'c']):
    axes[0].axhline(y=y, color=c, linestyle='-', linewidth=0.5)
    axes[1].axhline(y=y, color=c, linestyle='-', linewidth=0.5)

: 